In [ ]:
# найти все кейсы в выбранной папке содержащие все зубы 

In [2]:
import os
import shutil
import glob
from subprocess import run, PIPE
import sys
# from server.inference.model_inference import get_pediction, get_predicted_transforms
from fastapi import Body
import glob 

from autosetup_ml.utils import *
from typing import List, Dict, Any
import numpy as np

In [9]:
oas_folder = r"E:\awsCollectedData1\anonim"
oas_files = glob.glob(os.path.join(oas_folder, "*.oas"))[:50]

In [ ]:
# найти все кейсы в выбранной папке содержащие все зубы 
oas_folder = r"E:\awsCollectedData1\anonim"
proper_cases = []
for file in oas_files:
    oas_fname = os.path.basename(file).replace(".oas", "")
    orthoCase = OrthoCase(file)
    for tooth_id_to_find in dw_teeth_nums14 + up_teeth_nums14:
        if not orthoCase.has_tooth(tooth_id_to_find):
            print(f"Case {case_id} does not have tooth {tooth_id_to_find}")
            break
    else:
        print(f"Case {case_id} has all teeth")
        proper_cases.append(case_id)
print(f"Found {len(proper_cases)} cases with all teeth in {oas_folder}")
proper_cases.sort()

Case 100310 has all teeth
Case 100497 has all teeth
Case 101390 has all teeth
Case 101690 has all teeth
Case 101922 has all teeth
Case 102014 has all teeth
Case 102074 has all teeth
Case 102254 has all teeth
Case 102650 has all teeth
Case 103217 has all teeth
Case 103258 has all teeth
Case 103439 has all teeth
Case 103455 has all teeth
Case 103482 has all teeth
Case 103635 has all teeth
Case 103748 has all teeth
Case 103779 has all teeth
Case 103813 has all teeth
Case 103931 has all teeth
Case 103958 has all teeth
Case 104118 has all teeth
Case 104121 has all teeth
Case 104194 has all teeth
Case 104250 has all teeth
Case 104303 has all teeth
Case 104314 has all teeth
Case 104406 has all teeth
Case 104643 has all teeth
Case 104954 has all teeth
Case 105027 has all teeth
Case 105073 has all teeth
Case 105164 has all teeth
Case 105389 has all teeth
Case 105618 has all teeth
Case 105637 has all teeth
Case 105714 has all teeth
Case 105923 has all teeth
Case 106135 has all teeth
Case 106265 

In [22]:
class SolverData:
    def __init__(self, solverTeeth, jawType):
        self.solverTeeth = solverTeeth
        self.solverTeethMap = {tooth.id(): tooth.index() for tooth in solverTeeth}
        self.algo = PyResolveCollisionAlgo(G.doc)
        self.nDOF = 6 # degrees of freedom per tooth - 3 for rotation, 3 for translation
        self.nVar = len(self.solverTeeth)
        self.frame = 31
        self.t2 = G.t2Stage
        self.jawType = jawType
        self.occToJaw = G.occlusalToJawTransforms[jawType]
        self.arch = G.archModels[jawType]
        self.xformsT1 = [tooth.transform(0) for tooth in self.solverTeeth]
        self.xformsT2 = [tooth.transform(tooth.t2()) for tooth in self.solverTeeth]
        self.varsT1 = np.zeros((self.nDOF, self.nVar))
        self.varsT2 = np.zeros((self.nDOF, self.nVar))
        self.vars = self.varsT1.copy()
        for i,tooth in enumerate(self.solverTeeth):
             self.varsT1[:,i] = self.varsFromXform(self.xformsT1[i])
             self.varsT2[:,i] = self.varsFromXform(self.xformsT2[i])
        #self.blocks = [ProclineBlock(self, 0, 31, [42, 41, 31, 32]),
        #               IdleBlock(self, 0, 31, [47, 46, 45, 44, 43, 33, 34, 35, 36, 37])]
        self.blocks = [ProclineBlock(self, 0, 31, [33, 32, 31, 41, 42, 43]),
                       ExpandBlock(self, 0, 31, [44, 45, 46, 47]),
                       ExpandBlock(self, 0, 31, [37, 36, 35, 34])]
    
    def toothIndex(self, num):
        return self.solverTeethMap[num]
    
    def toothByNum(self, num):
        idx = self.solverTeethMap[num]
        return self.solverTeeth[idx]

    def toothByIndex(self, idx):
        return self.solverTeeth[idx]
    
    def xformFromVars(self, vars):
        basis = getLocalBasis(self.arch, vars[Parameter])
        euler = PyVector.fromArray(vars[0:3])
        m = PyMatrix() # local to arch
        m.addRotationComponent(euler[0], m.getAxis(Axis.Y))
        m.addRotationComponent(euler[1], m.getAxis(Axis.X))
        m.addRotationComponent(euler[2], m.getAxis(Axis.Z))
        m.setTranslationComponent(PyVector(0.0, vars[Buccal], vars[Occlusal]))
        archToLocal = getArchBasisTransform(basis) * PyRigidTransform.fromMatrix(m)
        return self.occToJaw * archToLocal

    def varsFromXform(self, xform):
        vars = np.zeros((self.nDOF,))
        toothToOcc = self.occToJaw.inverse() * xform
        basis = projectOnSpline(self.arch, toothToOcc.transformP(Origin))
        euler = getEulerAnglesFromLocalBasis(basis, toothToOcc)
        vars[0:3] = euler.asArray()
        vars[Parameter] = basis.u
        basisXform = getArchBasisTransform(basis).inverse()
        localToArch = basisXform * toothToOcc
        vars[Buccal] = localToArch.translation[1]
        vars[Occlusal] = localToArch.translation[2]
        return vars

    def localPointInArch(self, vars, p):
        toothToOcc = self.occToJaw.inverse() * self.xformFromVars(vars)
        basis = getLocalBasis(self.arch, vars[Parameter])
        basisXform = getArchBasisTransform(basis).inverse()
        return (basisXform * toothToOcc).transformP(p)

    def interp(self, lmbd, dof, idx):
        return (1 - lmbd) * self.varsT1[dof,idx] + lmbd * self.varsT2[dof,idx]

    def calcSpace(self, idx1, idx2, xform1, xform2):
        tooth1 = self.solverTeeth[idx1]
        tooth2 = self.solverTeeth[idx2]
        return self.algo.computeTeethDistance(tooth1, tooth2, xform1, xform2)

    def calcSpaces(self, idxs, xforms):
        n = len(idxs)
        spaces = np.zeros((n - 1,))
        for i in range(n - 1):
            spaces[i] = self.calcSpace(idxs[i], idxs[i + 1], xforms[i], xforms[i + 1])
        return spaces

    def fun(self, x):
        vars = x.reshape((self.nDOF, self.nVar))
        s = 0.0
        for block in self.blocks:
            s += block.fun(vars)
        return s
    
    def cons(self):
        cons = []
        for block in self.blocks:
            blockCons = block.cons()
            if blockCons is not None:
                cons.append(blockCons)
        return cons

    def bounds(self):
        lb = np.full((self.nDOF, self.nVar), -np.inf)
        ub = np.full((self.nDOF, self.nVar), np.inf)
        for block in self.blocks:
            block.initBounds(lb, ub)
        return scipy.optimize.Bounds(lb.flatten(), ub.flatten())

    def x0(self):
        vars = np.zeros((self.nDOF, self.nVar))
        for i,tooth in enumerate(self.solverTeeth):
            vars[:,i] = self.varsFromXform(transforms[self.frame][tooth.id()])
        x = vars.flatten()
        # Apply constraints
        bnds = self.bounds()
        for i in range(x.shape[0]):
            if x[i] < bnds.lb[i]: x[i] = bnds.lb[i]
            elif x[i] > bnds.ub[i]: x[i] = bnds.ub[i]
        return x
    
    def setX(self, x, interpolate=False):
        self.vars = x.reshape((self.nDOF, self.nVar))
        for idx,tooth in enumerate(self.solverTeeth):
            transforms[self.frame][tooth.id()] = self.xformFromVars(self.vars[:,idx])
        if not interpolate: return
        # Linear interpolation from 0 to frame
        for i in range(1, self.frame):
            for tooth in self.solverTeeth:
                n = tooth.id()
                transforms[i][n] = sclerp(transforms[0][n], transforms[self.frame][n], i / self.frame)
        for i in range(self.frame + 1, self.t2):
            for tooth in self.solverTeeth:
                n = tooth.id()
                transforms[i][n] = sclerp(transforms[self.frame][n], transforms[self.t2][n], 
                                          (i - self.frame) / (self.t2 - self.frame))
    
    def saveToTeeth(self):
        for idx,tooth in enumerate(self.solverTeeth):
            # Remove intermediate keyframes
            stages = [tooth.getFrame(i).timeAsInt() for i in range(tooth.keyFrameCount())]
            for stage in stages:
                if stage > 0 and stage < self.t2:
                    tooth.removeKeyFrame(stage)
        for block in self.blocks:
            block.createKeyFrames()
        saveSolverTeethToJaw(self.solverTeeth, self.t2, G.tp.GetJaw(self.jawType))
    
    def printVars(self):
        fmt = "{:2d} {:5.2f} {:5.2f} {:5.2f} {:5.2f} {:5.2f} {:5.2f}"
        d = self.vars - self.varsT1
        print("N  ANGUL INCLN ROTAT PARAM BUCCL OCCLS")
        print("--------------------------------------")
        for idx,tooth in enumerate(self.solverTeeth):
            print(fmt.format(tooth.id(), d[0,idx], d[1,idx], d[2,idx], d[3,idx], d[4,idx], d[5,idx]))

In [24]:
Identity = PyRigidTransform()
Origin = PyPoint()

class G: 
    showAbsoluteRotation = False
    showSpaces = False
    animate = False
    needUpdate = False
    intermediate_result = None

       
oas_folder = r"E:\awsCollectedData1\anonim"
oas_fname = os.path.join(oas_folder, "103931.oas")

G.doc = PyOrthoDoc()
G.doc.OpenCase(oas_fname)
createControllers(G.doc) # calculates archforms
G.case = case = G.doc.getCase()
G.tp = tp = G.doc.GetActiveTreatmentPlan()
G.t2Stage = t2Stage = max([tp.GetJaw(jaw).GetT2() for jaw in JawType])
G.jawTransforms = [tp.GetJaw(jawType).relativeTransform(0) for jawType in JawType]
G.occlusalToJawTransforms = [tp.getJawOcclusalCS(jawType) for jawType in JawType]
G.archModels = [getArchSplineT1(case, tp, jawType) for jawType in JawType]


def onSolve():
    jaw = G.tp.GetJaw(JawType.Mandible)
    solverTeeth = createSolverTeeth(jaw)
    selection = set([41, 31])
    selected = []
    for tooth in solverTeeth:
        if tooth.id() in selection: selected.append(tooth)
    G.solverData = SolverData(solverTeeth, jaw.getJawType())
    solver = Solver(G.solverData)
    solver.solve()
onSolve()

NameError: name 'Parameter' is not defined